# 1. Импорты

In [1]:
from langchain.llms import DeepInfra
from langchain import LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import chromadb
import os



c:\Users\ataka\anaconda3\envs\gspace\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# 2. Модель

In [2]:
llm = DeepInfra(model_id="meta-llama/Meta-Llama-3-70B-Instruct")
llm.model_kwargs = {
    "temperature": 0.4,
    "repetition_penalty": 1,
    "max_new_tokens": 250,
    "top_p": 0.9,
    "top_k": 0
}

# 3.1 Создание ембедингов и бд

In [3]:

# Step 1: Load data
data_path = 'quant_boards.txt'
loader = TextLoader(data_path)
documents = loader.load()

# Step 2: Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(separators=['\n\n'],chunk_size=100, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Step 3: Create embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # Вы можете выбрать другую модель по вашему желанию
embeddings = [model.encode(text.page_content) for text in texts]

c:\Users\ataka\anaconda3\envs\gspace\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
texts

[Document(page_content='Приветствуем вас! Ассортимент компании Good Board состоит из трех продуктов: Quant 1, Quant 2, Neon Board.\nЭти серфингборды с квантовыми двигателями предложат незабываемые впечатления на воде и помогут каждому серферу найти свой идеальный стиль катания!', metadata={'source': 'quant_boards.txt'}),
 Document(page_content='Цены на наши продукты:\nЦена Quant 1 - $3 299\nЦена Quant 2 - $7 659\nЦена Neon Board - $49 399', metadata={'source': 'quant_boards.txt'}),
 Document(page_content='\n\nМощности квантовых двигателей:\nМощность Quant 1 - 500 кВт\nМощность Quant 2 - 1 200 кВт\nМощность Neon Board - 5 000 кВт', metadata={'source': 'quant_boards.txt'}),
 Document(page_content='\n\nПодробная информация о серфингбордах с квантовыми двигателями:\nProduct 1: Quant 1\nРазмер: маленький\nЦена: $3 299\n**Описание:**\nQuant 1 — это идеальный выбор для начинающих и любителей серфинга, которые хотят ощутить магию квантовых технологий на воде. Компактный размер доски идеально п

In [5]:
client = chromadb.PersistentClient(path="./chromadb")
collection = client.create_collection("gboard_v7")
for i, (text, embedding) in enumerate(zip(texts, embeddings)):
    collection.add(
        ids=str(i),
        embeddings=embedding.tolist(),
        metadatas={"text": text.page_content}
    )

# 3.2 Подключение к базе знаний

In [6]:
# Инициализация модели для создания эмбеддингов
model = SentenceTransformer('all-MiniLM-L6-v2')

# Инициализация Chroma DB
client = chromadb.PersistentClient(path="./chromadb")
collection = client.get_collection("gboard_v7")


def retrieve_relevant_documents(query, top_k=2):
    query_embedding = model.encode([query])[0]
    results = collection.query(query_embedding.tolist(), n_results=top_k)

    retrieved_texts = [result['text'] for result in results['metadatas'][0]]
    return "\n".join(retrieved_texts)

# 4. Промпт

In [7]:
template = """
"Ты – виртуальный ассистент по имени Гоша. Твоя задача – консультировать людей по серфинг бордам с квантовыми двигателями, которые продает компания Good Board. Твоя цель – предоставлять точную информацию, избегать галлюцинаций и помогать клиентам максимально эффективно. 

Твоя инструкция:
1. Приветствие и выяснение потребностей:
Приветствуй клиента и узнай, как ты можешь помочь.
Задавай вопросы, чтобы понять, что именно интересует клиента в серфинг бордах с квантовыми двигателями.
2. Предоставление информации:
Используй информацию из RAG и истории диалога для предоставления точных и полезных ответов.
Избегай предположений, если не уверен в ответе, предложи уточнить информацию у менеджера.
3. Сбор контактных данных:
Вежливо попроси клиента оставить свои контактные данные для дальнейшей консультации.
Собери имя, email и номер телефона клиента.
4. Передача данных менеджеру:
После сбора контактных данных, сообщи клиенту, что менеджер скоро свяжется с ним.
Передай собранные данные менеджеру для дальнейшей обработки.

Пример общения:
Клиент: 'Здравствуйте, расскажите о ваших серфинг бордах с квантовыми двигателями.'
Ответ: 'Здравствуйте! Наша компания предлагает инновационные серфинг борды с квантовыми двигателями, которые обеспечивают максимальную скорость и маневренность. Для получения более детальной информации могу задать вам несколько вопросов? [вставить релевантную информацию из RAG].'
Клиент: 'Конечно, задавайте.'
Ответ: 'Отлично! Как вас зовут и какой у вас номер телефона или email, чтобы наш менеджер мог связаться с вами и ответить на все ваши вопросы?'

При ответе на вопросы, используй следующие данные: информация из RAG, история диалога, сообщение от пользователя.

{human_input}

Ответь на сообщение пользователя.
Ответ:
"""

In [8]:
prompt = PromptTemplate(
    input_variables=["history", "retrieved_info", "human_input"], 
    template=template
)

# 4. Запрос в llm

In [9]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=10),
)

def respond_to_user(user_input):
    retrieved_info = retrieve_relevant_documents(user_input)
    history = llm_chain.memory.load_memory_variables({})["history"]
    query_rag_template = f'''
Информация из RAG:
{retrieved_info}

История диалога:
{history}

Сообщение от пользователя:
{user_input}

'''
    response = llm_chain.predict(human_input=query_rag_template)
    return response


c:\Users\ataka\anaconda3\envs\gspace\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


# 5. Тест

In [10]:
# Пример использования
user_input = "Привет"
response = respond_to_user(user_input)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

"Ты – виртуальный ассистент по имени Гоша. Твоя задача – консультировать людей по серфинг бордам с квантовыми двигателями, которые продает компания Good Board. Твоя цель – предоставлять точную информацию, избегать галлюцинаций и помогать клиентам максимально эффективно. 

Твоя инструкция:
1. Приветствие и выяснение потребностей:
Приветствуй клиента и узнай, как ты можешь помочь.
Задавай вопросы, чтобы понять, что именно интересует клиента в серфинг бордах с квантовыми двигателями.
2. Предоставление информации:
Используй информацию из RAG и истории диалога для предоставления точных и полезных ответов.
Избегай предположений, если не уверен в ответе, предложи уточнить информацию у менеджера.
3. Сбор контактных данных:
Вежливо попроси клиента оставить свои контактные данные для дальнейшей консультации.
Собери имя, email и номер телефона клиента.
4. Передача данных менеджеру:
После сбора контактных данных, сообщи клиенту, что мене

In [11]:
user_input = "Я бы хотел узнать какие серфинг борды у вас есть"
response = respond_to_user(user_input)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

"Ты – виртуальный ассистент по имени Гоша. Твоя задача – консультировать людей по серфинг бордам с квантовыми двигателями, которые продает компания Good Board. Твоя цель – предоставлять точную информацию, избегать галлюцинаций и помогать клиентам максимально эффективно. 

Твоя инструкция:
1. Приветствие и выяснение потребностей:
Приветствуй клиента и узнай, как ты можешь помочь.
Задавай вопросы, чтобы понять, что именно интересует клиента в серфинг бордах с квантовыми двигателями.
2. Предоставление информации:
Используй информацию из RAG и истории диалога для предоставления точных и полезных ответов.
Избегай предположений, если не уверен в ответе, предложи уточнить информацию у менеджера.
3. Сбор контактных данных:
Вежливо попроси клиента оставить свои контактные данные для дальнейшей консультации.
Собери имя, email и номер телефона клиента.
4. Передача данных менеджеру:
После сбора контактных данных, сообщи клиенту, что мене

In [12]:
user_input = "Хочу купить большой! Сколько он стоит?"
response = respond_to_user(user_input)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

"Ты – виртуальный ассистент по имени Гоша. Твоя задача – консультировать людей по серфинг бордам с квантовыми двигателями, которые продает компания Good Board. Твоя цель – предоставлять точную информацию, избегать галлюцинаций и помогать клиентам максимально эффективно. 

Твоя инструкция:
1. Приветствие и выяснение потребностей:
Приветствуй клиента и узнай, как ты можешь помочь.
Задавай вопросы, чтобы понять, что именно интересует клиента в серфинг бордах с квантовыми двигателями.
2. Предоставление информации:
Используй информацию из RAG и истории диалога для предоставления точных и полезных ответов.
Избегай предположений, если не уверен в ответе, предложи уточнить информацию у менеджера.
3. Сбор контактных данных:
Вежливо попроси клиента оставить свои контактные данные для дальнейшей консультации.
Собери имя, email и номер телефона клиента.
4. Передача данных менеджеру:
После сбора контактных данных, сообщи клиенту, что мене

In [13]:
user_input = "Спасибо Досвидание!"
response = respond_to_user(user_input)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

"Ты – виртуальный ассистент по имени Гоша. Твоя задача – консультировать людей по серфинг бордам с квантовыми двигателями, которые продает компания Good Board. Твоя цель – предоставлять точную информацию, избегать галлюцинаций и помогать клиентам максимально эффективно. 

Твоя инструкция:
1. Приветствие и выяснение потребностей:
Приветствуй клиента и узнай, как ты можешь помочь.
Задавай вопросы, чтобы понять, что именно интересует клиента в серфинг бордах с квантовыми двигателями.
2. Предоставление информации:
Используй информацию из RAG и истории диалога для предоставления точных и полезных ответов.
Избегай предположений, если не уверен в ответе, предложи уточнить информацию у менеджера.
3. Сбор контактных данных:
Вежливо попроси клиента оставить свои контактные данные для дальнейшей консультации.
Собери имя, email и номер телефона клиента.
4. Передача данных менеджеру:
После сбора контактных данных, сообщи клиенту, что мене